In [1]:
import os
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from adcircpy import AdcircMesh
from adcircpy.mesh.base import Elements

In [2]:
# Paths to the input/output files
basedir = "/home/sbunya/GitHub/adcircutils/adcircutils/vewchannel/examples/example2"
f14file = os.path.join(basedir, "fort_with_vews.14")
f14file_out = os.path.join(basedir, "fort_without_vews.14")
vewfile_out = os.path.join(basedir, "vewstings_stripped.yaml")

In [ ]:
# This function removes a specific vew boundary from the mesh.
# It updates the node and element tables by replacing the bank nodes with their corresponding channel nodes.
# The function also updates the boundary table to reflect the changes and removes the specified vew boundary.
def strip_vewstring(mesh, ivewboundary):
    map_elem_node_next = {0: 1, 1: 2, 2: 0}

    boundaries = mesh.boundaries.to_dict().copy()
    vewboundaries = boundaries['64']
    
    node_elements = mesh.node_elements
    node_neighbors = mesh.node_neighbors

    vewboundary = vewboundaries[ivewboundary]

    bank_nodes = [int(vewboundary['node_id'][i][0]) for i in range(len(vewboundary['node_id']))]
    channel_nodes = [int(vewboundary['node_id'][i][1]) for i in range(len(vewboundary['node_id']))]

    # Making a new mesh without the vew boundary ---------------------------------------------------------
    # Prepare for making a new mesh
    nodes = mesh.nodes.copy()
    nodes_new = mesh.nodes.copy()
    elements_new = mesh.elements.elements.copy()
    boundaries_new = boundaries.copy()

    # Create a mapping table between the old and new node numbers
    idx = 1
    map_node = {}
    for i in nodes_new.index:
        if i in bank_nodes:
            idx_bank = bank_nodes.index(i)
            map_node[i] = channel_nodes[idx_bank]
        else:
            map_node[i] = idx
            idx += 1

    # Update node ids in the element table
    elements_new[['node_1', 'node_2', 'node_3']] = elements_new[['node_1', 'node_2', 'node_3']].replace(map_node)

    # Drop the bank nodes in the node table
    nodes_new = nodes_new.drop(index=bank_nodes)
    nodes_new.index = nodes_new.index.map(map_node)

    # Drop the targetted vew boundary
    boundaries_new['64'].pop(ivewboundary)

    # Update the node numbers in the boundary table
    for ibtype in boundaries.keys():
        for i in range(len(boundaries[ibtype])):
            # Drop the bank nodes in the boundary table
            if ibtype is None:
                dfbnd = pd.DataFrame(boundaries[ibtype][i]['node_id'])
                dfbnd.replace(map_node, inplace=True)
                boundaries_new[ibtype][i]['node_id'] = dfbnd.iloc[:,0].values
            elif ibtype.endswith('4'):
                dfbnd = pd.DataFrame(boundaries[ibtype][i]['node_id'])
                dfbnd = dfbnd.replace(map_node)
                boundaries_new[ibtype][i]['node_id'] = list(zip(dfbnd.iloc[:,0].values, dfbnd.iloc[:,1].values))
            else:
                dfbnd = pd.DataFrame(boundaries[ibtype][i]['node_id'])
                dfbnd.replace(map_node, inplace=True)
                boundaries_new[ibtype][i]['node_id'] = dfbnd.values.tolist()
                
    new_mesh = AdcircMesh(nodes=nodes_new, elements=elements_new, boundaries=boundaries_new)

    triangles = mesh.elements.elements.iloc[:, 1:4]

    # Making a vewstring ---------------------------------------------------------------------
    # Reverse node strings if it's not seeing the bank side on the right.
    first_node_elems = node_elements[bank_nodes[0]]
    second_node_elems = node_elements[bank_nodes[1]]
    common_elems = list(set(first_node_elems) & set(second_node_elems))
    if len(common_elems) != 1:
        raise ValueError("The number of common elements between the two bank nodes is not 1")
    elem_nodes = list(triangles.loc[common_elems[0]])
    index = elem_nodes.index(bank_nodes[1])
    if elem_nodes[map_elem_node_next[index]] != bank_nodes[0]:
        bank_nodes.reverse()
        channel_nodes.reverse()

    # If this node string is not a loop, find the common node between the bank and channel nodes at the ends.
    # The common nodes should be added to the node string.
    if bank_nodes[0] != bank_nodes[-1]:
        nneigh_bank = node_neighbors[bank_nodes[0]]
        nneigh_channel = node_neighbors[channel_nodes[0]]
        common_node = list(set(nneigh_bank) & set(nneigh_channel))
        if len(common_node) != 1:
            raise ValueError("The number of common nodes between the bank and channel nodes is not 1")
        bank_nodes.insert(0, common_node[0])
        channel_nodes.insert(0, common_node[0])
        
        nneigh_bank = node_neighbors[bank_nodes[-1]]
        nneigh_channel = node_neighbors[channel_nodes[-1]]
        common_node = list(set(nneigh_bank) & set(nneigh_channel))
        if len(common_node) != 1:
            raise ValueError("The number of common nodes between the bank and channel nodes is not 1")
        bank_nodes.append(common_node[0])
        channel_nodes.append(common_node[0])
        
    # Create a vewstring
    vewstring = []
    for i in range(len(bank_nodes)):
        vewstring.append({'node_id': int(map_node[channel_nodes[i]]),
                        'x': float(nodes.loc[bank_nodes[i], 'x']),
                        'y': float(nodes.loc[channel_nodes[i], 'y']),
                        'bank_elevation': float(nodes.loc[bank_nodes[i], 'value_1']),
                        'bank_mannings_n': 0.03})
               
    return new_mesh, vewstring


# This function iterates through all vew boundaries in the mesh and removes them one by one
# by calling the strip_vewstring function. It returns a new mesh with all vew boundaries stripped.
def strip_vewstrings(mesh):
    boundaries = mesh.boundaries.to_dict().copy()
    vewboundaries = boundaries['64']

    vewstrings = []
    for ivewboundary in range(len(vewboundaries)):
        mesh, vewstring = strip_vewstring(mesh, 0)
        vewstrings.append(vewstring)
    
    vewstrings = {'vewstrings': vewstrings}

    return mesh, vewstrings

# Load the mesh file
mesh = AdcircMesh.open(f14file)

# Strip all vew boundaries from the mesh
mesh_new, vewstrings = strip_vewstrings(mesh)
mesh_new.description = "Generated by strip_vews_from_mesh"

# Save the new mesh file
f14file_out = os.path.join(basedir, "fort_vews_stripped.14")
mesh_new.write(f14file_out, overwrite=True)

# Write the YAML output to a file
vewfile_out = os.path.join(basedir, "vewstings_stripped.yaml")
with open(vewfile_out, 'w') as f:
    yaml.dump(vewstrings, f, sort_keys=False)

/home/sbunya/GitHub/adcircpy/adcircpy/mesh/parsers/grd.py:277: UserWarning: File does not contain CRS information and no CRS was given: "/home/sbunya/GitHub/adcircutils/adcircutils/vewchannel/examples/example2/fort_with_vews.14"
  warnings.warn(
